<a href="https://colab.research.google.com/gist/MarkCreightonQueens/c1875f4429be64b4e6ca52a36bd3e263/machine-learning-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine Learning Data:

Goal is to input relevant FOMC documentation and otbtain text data/sentences that are readable by machine learning algorithms. 
 



# Setting up the environment

Import of needed tools

Import of data

Toggling on the GPU


In [ ]:
#installing tools
!pip install transformers 
!pip install wget
!pip install tika



     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 21.0MB/s 
     |████████████████████████████████| 3.0MB 20.3MB/s 
     |████████████████████████████████| 890kB 23.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e96703c2e53da96fd717afdd959b3549d90183a990e1e782aa3031c2188ecde0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=40db3ee44925d2296ac68110a8d8fc49149d80ba4e185d3cbbfb59712d55e978
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=e041ea6f185af9a5b042b89875b37879c6ae04c0e7022b4f4eaacf1bc2ca0570
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b

In [ ]:
#Importing tools
import pandas as pd
import numpy as np
from tika import parser
#Note: need to run older version of tensor flor for BERT to work. 
%tensorflow_version 1.x
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import os
import shutil
import glob
import nltk
nltk.download('punkt')
import wget
import urllib.request
import re
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import matplotlib.pyplot as plt
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import datetime
import pickle
from sklearn.naive_bayes import MultinomialNB


TensorFlow 1.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at:{}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at:/device:GPU:0


In [ ]:
#Determining what GPU is available for use
if torch.cuda.is_available():
  device = torch.device("cuda")
  print ('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else: 
  print('No GPU available, using CPU instead.')
  device = torch.device("CPU")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Data Import

Importing FOMC meeting minutes and transcripts

TODO: get Amazon data for training purposes. 

In [ ]:
#Extracting the data from the zip file
shutil.unpack_archive('/content/drive/My Drive/Thesis/FOMC Documentation.zip')
print('Complete.')

Complete.


# Data Cleaning 

Part 1: Cleaning the minutes

Note: in 2007 the minute format changed, as a result of this two separate approaches are necessary to obtain the same final result

Part 2: Cleaning the transcripts

This may be somewhat involved...

In [ ]:
#Declaring functions that will be used to clean the data. 

#remove leading space in entries to allow for easier combination/interpretation
#combined if ends in not '.' and starts with month.  


#function to remove empty entries in the text array
def remove_empties(string_list):
    output = []
    for string in string_list:
        # This if statement will see spaces as empty, i.e ' ' will be removed
        if string.strip():
            output.append(string)
    return output

#function to remove 1 character strings.


#function to remove text break new line notation in the text array 
 
def remove_line_breaks(string_list):
    output = []
    for string in string_list:
        if '-\n' in string:
            output.append(string.replace('-\n',''))
        else:
            output.append(string)
    return output
 
#function to remove new line notation in the text array
def remove_whitespace(string_list):
    output = []
    for string in string_list:
        if string != re.sub('\n','', string):
            output.append(re.sub('\n','', string))
        else:
            output.append(string)
    return output

#Functions to recombine paragraphs that were erroneously separated
def combine_erroneous_separations_if_word_break(string_list):
    output = []
    string_list = string_list.copy()
    while string_list:
        #Pops the first string off the list
        first_string = string_list.pop(0)
        if first_string[-1] == '-':
            #Pops the next string off
            second_string = string_list.pop(0)
            #Concatenates them together, removing the end dash
            output.append(first_string[:-1] + second_string)
            continue
        else:
            output.append(first_string)
    return output

#TODO: Double check this, some paragraphs are not being recombined despite meeting the criteria AFAIK
#Function to recombine erroneously separated paragraphs due to page breaks
def combine_erroneous_separations_if_page_break(string_list):
    output = []
    string_list = string_list.copy()
    while string_list:
        #Pops the first string off the list
        first_string = string_list.pop(0)
        if first_string[-1] != '.' and string_list:
            #Pops the next string off
            second_string = string_list.pop(0)
            if second_string[0].islower() or second_string[0] == 'I' or second_string[0:6] == 'Reserve':
            #Concatenates them together, removing the end dash
                    results = first_string +' '+second_string
                    output.append(results)
                    continue
            output.append(first_string)
            output.append(second_string)
        else:
            output.append(first_string)
    return output

#function to remove double spaces
def remove_double_spaces(string_list):
    output = []
    for string in string_list:
        results1 = string
        results2 = re.sub('  ',' ', string)
        if results1 != results2:
            while results1 != results2:
                results1 = re.sub('  ',' ', results1)
                results2 = re.sub('  ',' ', results2)
                if results1 == results2:
                    output.append(results1)
        else:
            output.append(results1)
    return output

#remove space at the end of the entry
def remove_end_spaces(string_list):
    output = []
    for string in string_list:
        if string[-1] == ' ':
            string = string[0:-1]
        output.append(string)
    return output

#re-add apostrophes
def add_apostrophes(string_list):
    output = []
    for string in string_list:
        results = string
        results = re.sub('â€œ','\"', results)
        results = re.sub('â€™','\'', results)
        output.append(results) 
    return output

#Remove page references. Note: Run remove_double_spaces
def remove_page_references(string_list):
    output = []
    for string in string_list:
        results = re.sub(r'Page \d+','', string)
        output.append(results) 
    return output

#Remove FOMC formatting
def remove_FOMC_formatting(string_list):
    output = []
    for string in string_list:
        
        #There is no easy way to detext these without risking the elimination of useful information, so I'm using an exhaustive approach to avoid false positives. 
        results = string
        #Single day events
        #Full spelling
        results = re.sub('Minutes of the Federal Open Market Committee January \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee February \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee March \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee April \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee May \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee June \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee July \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee August \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee September \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee October \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee November \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee December \d+, \d+','', results)
        #Full spelling with comma
        results = re.sub('Minutes of the Federal Open Market Committee, January \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, February \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, March \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, April \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, May \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, June \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, July \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, August \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, September \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, October \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, November \d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, December \d+, \d+','', results)   
        #Alternative spelling
        results = re.sub('Minutes of Federal Open Market Committee October 30-31, 2007','', results)
        results = re.sub('Minutes of Federal Open Market Committee January \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee February \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee March \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee April \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee May \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee June \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee July \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee August \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee September \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee October \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee November \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee December \d+, \d+','', results)
        #Alternative spelling with commas
        results = re.sub('Minutes of Federal Open Market Committee, October 30-31, 2007','', results)
        results = re.sub('Minutes of Federal Open Market Committee, January \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, February \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, March \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, April \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, May \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, June \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, July \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, August \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, September \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, October \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, November \d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, December \d+, \d+','', results)
        #Minutes of the meeting
        results = re.sub('Minutes of Federal Open Market Committee, October 30-31, 2007','', results)
        results = re.sub('Minutes of the Meeting of January \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of February \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of March \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of April \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of May \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of June \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of July \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of August \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of September \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of October \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of November \d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of December \d+, \d+','', results)       
        #FOMC spelling
        results = re.sub('FOMC Minutes, October 30-31, 2007','', results)
        results = re.sub('FOMC Minutes, January \d+, \d+','', results)
        results = re.sub('FOMC Minutes, February \d+, \d+','', results)
        results = re.sub('FOMC Minutes, March \d+, \d+','', results)
        results = re.sub('FOMC Minutes, April \d+, \d+','', results)
        results = re.sub('FOMC Minutes, May \d+, \d+','', results)
        results = re.sub('FOMC Minutes, June \d+, \d+','', results)
        results = re.sub('FOMC Minutes, July \d+, \d+','', results)
        results = re.sub('FOMC Minutes, August \d+, \d+','', results)
        results = re.sub('FOMC Minutes, September \d+, \d+','', results)
        results = re.sub('FOMC Minutes, October \d+, \d+','', results)
        results = re.sub('FOMC Minutes, November \d+, \d+','', results)
        results = re.sub('FOMC Minutes, December \d+, \d+','', results)
        #FOMC for spelling
        results = re.sub('FOMC Minutes for October 30-31, 2007','', results)
        results = re.sub('FOMC Minutes for January \d+, \d+','', results)
        results = re.sub('FOMC Minutes for February \d+, \d+','', results)
        results = re.sub('FOMC Minutes for March \d+, \d+','', results)
        results = re.sub('FOMC Minutes for April \d+, \d+','', results)
        results = re.sub('FOMC Minutes for May \d+, \d+','', results)
        results = re.sub('FOMC Minutes for June \d+, \d+','', results)
        results = re.sub('FOMC Minutes for July \d+, \d+','', results)
        results = re.sub('FOMC Minutes for August \d+, \d+','', results)
        results = re.sub('FOMC Minutes for September \d+, \d+','', results)
        results = re.sub('FOMC Minutes for October \d+, \d+','', results)
        results = re.sub('FOMC Minutes for November \d+, \d+','', results)
        results = re.sub('FOMC Minutes for December \d+, \d+','', results)

        #Multi day events
        #Full spelling
        results = re.sub('Minutes of the Federal Open Market Committee January \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee February \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee March \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee April \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee May \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee June \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee July \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee August \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee September \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee November \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee December \d+\-\d+, \d+','', results)
        #Full spelling with comma
        results = re.sub('Minutes of the Federal Open Market Committee, January \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, February \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, March \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, April \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, May \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, June \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, July \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, August \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, September \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, November \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Federal Open Market Committee, December \d+\-\d+, \d+','', results)   
        #Alternative spelling
        results = re.sub('Minutes of Federal Open Market Committee October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee January \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee February \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee March \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee April \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee May \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee June \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee July \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee August \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee September \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee November \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee December \d+\-\d+, \d+','', results)
        #Alternative spelling with commas
        results = re.sub('Minutes of Federal Open Market Committee, October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, January \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, February \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, March \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, April \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, May \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, June \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, July \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, August \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, September \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, November \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of Federal Open Market Committee, December \d+\-\d+, \d+','', results)
        #Minutes of the meeting
        results = re.sub('Minutes of Federal Open Market Committee, October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of January \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of February \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of March \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of April \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of May \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of June \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of July \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of August \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of September \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of October \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of November \d+\-\d+, \d+','', results)
        results = re.sub('Minutes of the Meeting of December \d+\-\d+, \d+','', results)       
        #FOMC spelling
        results = re.sub('FOMC Minutes, October \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, January \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, February \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, March \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, April \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, May \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, June \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, July \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, August \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, September \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, October \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, November \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes, December \d+\-\d+, \d+','', results)
        #FOMC for spelling
        results = re.sub('FOMC Minutes for October \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for January \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for February \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for March \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for April \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for May \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for June \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for July \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for August \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for September \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for October \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for November \d+\-\d+, \d+','', results)
        results = re.sub('FOMC Minutes for December \d+\-\d+, \d+','', results) 
        #Old formatting
        results = re.sub('$\d+/\d+/\d+ FRB(.*?)\d+/\d+^','', results)
        #6/17/2020 FRB: FOMC Minutes--January 31-February 1, 1995 https://www.federalreserve.gov/fomc/MINUTES/1995/19950201min.htm 1/14
        output.append(results) 
    return output

#Function to remove page reference
def remove_page_references(string_list):
    output = []
    for string in string_list:
        results = string   
        results = re.sub('Minutes of the Meeting of(.*?)Page \d+','', results)
        results = re.sub('Page \d+(.*?)Federal Open Market Committee','', results)
        
        output.append(results) 
    return output

#Function to remove entries comprised soley of digits
def remove_digit_only_entries(string_list):
    output = []
    for string in string_list:
        if re.match('^\d+$', string):
            results = ''
            output.append(results)
        else:
            output.append(string)
    return output

#Function to remove entries that correspond to d.dd.d
def remove_d_dd_d_entries(string_list):
    output = []
    for string in string_list:
        if re.match('^\d\.\d+\.\d$', string):
            results = ''
            output.append(results)
        else:
            output.append(string)
    return output

#Function to remove the formatting on tables that gets captured in the parser
def remove_table_formatting(string_list):
    output = []
    for string in string_list:
        if (re.match('^Number of participants$', string)) or (re.match('^Number of Participants$', string)) or (re.match('^Percentage range$', string)) or (re.match('^Percent range$', string)) or (re.match('^Longer run$', string)) or (re.match('^January projections$', string)) or (re.match('^February projections$', string)) or (re.match('^March projections$', string)) or (re.match('^April projections$', string)) or (re.match('^May projections$', string)) or (re.match('^June projections$', string)) or (re.match('^July projections$', string)) or (re.match('^August projections$', string)) or (re.match('^September projections$', string)) or (re.match('^October projections$', string)) or (re.match('^November projections$', string)) or (re.match('^December projections$', string))  or (re.match('^January Projections$', string)) or (re.match('^February Projections$', string)) or (re.match('^March Projections$', string)) or (re.match('^April Projections$', string)) or (re.match('^May Projections$', string)) or (re.match('^June Projections$', string)) or (re.match('^July Projections$', string)) or (re.match('^August Projections$', string)) or (re.match('^September Projections$', string)) or (re.match('^October Projections$', string)) or (re.match('^November Projections$', string)) or (re.match('^December Projections$', string)):
            results = ''
            output.append(results)
        else:
            output.append(string)
    return output


#Function ro remove strings that consist of a single character
def remove_single_digit_strings(string_list):
    output = []
    for string in string_list:
        if (re.match('^.$', string)):
            results = ''
            output.append(results)
        else:
            output.append(string)
    return output

#Function to remove strings with no letter characters
def remove_strings_without_letters(string_list):
    output = []
    for string in string_list:
        if re.search('[a-zA-Z]', string):
              output.append(string)
        else:
            results = ''
            output.append(results)
    return output

#Function to remove leading spaces in an entry
#Note: Run this after removing double spaces
def remove_leading_space(string_list):
    output = []
    string_list = string_list.copy()
    for string in string_list:
        if string[0] == ' ':
            results = string[1:]
            output.append(results)
        else:
            output.append(string)
    return output

#Function to remove formatting from old style documents
def remove_old_style_formatting(string_list):
    output = []
    for string in string_list:
        #There is no easy way to detext these without risking the elimination of useful information, so I'm using an exhaustive approach to avoid false positives. 
        results = string
        #Single day events
        #Full spelling
        results = re.sub('^https(.*?)\d$','', results)
        output.append(results)
    return output

def remove_old_style_date_formatting(string_list):
    output = []
    for string in string_list:
        #There is no easy way to detext these without risking the elimination of useful information, so I'm using an exhaustive approach to avoid false positives. 
        results = string
        #Single day events
        #Full spelling
        results = re.sub('^\d+/\d+/\d+(.*?)FRB(.*?)\d+$','', results)
        output.append(results)
    return output


#Function to remove the formatting on tables that gets captured in the parser
def remove_meta_data(string_list):
    output = []
    for string in string_list:
        if (re.match('ASCII(.*?)False$', string) or re.match('CreateJDFFile(.*?)setpagedevice$', string)) :
            results = ''
            output.append(results)
        else:
            output.append(string)
    return output


print('Complete.')


Complete.


In [ ]:
#Variables refrenced
#Minimum length that a paragraph can take:
#Selected to eliminate superfluous paragraphs that slip through the text cleaning. 
min_length_para = 8

#Minimum length that a sentence can take
#Selected to weed out stump stentences that have slipped through. 
min_length_sent = 2

#Maximum length that a sentence can take
#Selected due to model constaints, processing time increases exponentially as max length increases. 

max_length_sent = 128


In [ ]:
#Obtaining the minutes information for the new format

os.chdir('/content/FOMC Documentation/All Minutes')

#Defines a function that obtains the location of the desired text and saves it
def read_text(path):
    f = open(path, "r")
    return f.read()

def get_file_paths():
  return glob.glob('fomcminutes*.pdf')

get_file_paths()

#creates dataframe to store the text in for cleaning
def create_entry(ID,path,raw_text):
  return {'ID': ID, 'path': path, 'raw_text': raw_text}

#filling the empty dataframe
end_result = pd.DataFrame(columns=['ID','path','raw_text'])

count = 0
for path in get_file_paths():
    #gets text
    parsed = parser.from_file(path)
    raw_text = parsed['content']

    #Stores result as row = count
    end_result.loc[count] = create_entry(count,path,raw_text)

    #increments counter
    count = count+1

#sort the documents by date before applying functions

#function to get the date string from the file path.
def date_string(string_list):
    output = []
    for string in string_list:
          results = re.findall('\d+', string)
          results = "".join(results)
          output.append(results)
    return output

dates_temp = pd.Series(end_result['path'])

dates_temp = dates_temp.apply(date_string)
dates_temp = dates_temp.str.join("")

end_result['dates_temp'] = dates_temp


#TODO: 
#function to reformat datestring to mm/dd/yyyy
dates = []
for string in dates_temp:
    year = string[0:4]
    month = string[4:6]
    #if month[0] == '0':
        #month = month[1]
    day = string[6:8]
    #if day[0] == '0':
        #day = day[1]
    results = month + '/' + day + '/' + year
    dates.append(results)

end_result['dates'] = dates

end_result = end_result.sort_values(by = ['dates_temp'])

end_result['text'] = end_result['raw_text'].str.split('\n\n')

#testing
test1 = end_result['raw_text']
test2 = end_result['text']

#print(test1.apply(lambda x : 'The recent information on inflation was seen'  in x ).sum() > 0)
#print(test1.head())
#print(test1.shape)
test2 = test2.explode()
test2 = test2.loc[test2 !='']
test2 = test2.dropna()

outfile = 'test2.csv'
test2.to_csv(outfile)
test3 = pd.read_csv('test2.csv')
test3 = test3.dropna()
test3 = pd.Series(test3['text'])
test2 = test2.reset_index(drop = True)

#print(test2.apply(lambda x : 'The recent information on inflation was seen'  in x ).sum() > 0)
#print(test2.head())
#print(test2.shape)
#print(test3.apply(lambda x : 'The recent information on inflation was seen'  in x ).sum() > 0)
#print(test3.head())
#print(test3.shape) 


#is_equivalent = test2.eq(test3)
#print(is_equivalent.sum())
#\testing

end_result = end_result.set_index(end_result['dates_temp'], drop = True)

end_result['ID'] = np.arange(len(end_result))

text = pd.Series(end_result['text'])

#testing

#/testing

#testing
print('Before applying cleaning fuctions:')
test4 = text
test4 = test4.explode()
test4 = test4.loc[test4!='']
test4 = test4.reset_index(drop=True)
print(test4.head())
print(test4.shape)
print(test4.apply(lambda x : 'The recent information on inflation was seen'  in x ).sum() > 0)

#\testing 

#applying the gaggle of functions
text = text.apply(remove_digit_only_entries)
text = text.apply(remove_empties)
#TODO: combine if ends in not '.' and starts with month.  
text = text.apply(remove_double_spaces)
text = text.apply(remove_leading_space)
text = text.apply(remove_empties)
text = text.apply(combine_erroneous_separations_if_word_break)
text = text.apply(remove_double_spaces)
text = text.apply(remove_leading_space)
text = text.apply(remove_empties)
text = text.apply(combine_erroneous_separations_if_page_break) 
text = text.apply(remove_line_breaks)
text = text.apply(remove_whitespace)
text = text.apply(remove_page_references) 
text = text.apply(remove_digit_only_entries)
text = text.apply(remove_FOMC_formatting)
text = text.apply(remove_double_spaces)
text = text.apply(remove_empties)
text = text.apply(remove_end_spaces)
text = text.apply(add_apostrophes)
text = text.apply(remove_table_formatting)
text = text.apply(remove_strings_without_letters)
text = text.apply(remove_empties)
text = text.apply(remove_double_spaces)
text = text.apply(remove_leading_space)
test = text.apply(remove_meta_data)
text = text.apply(remove_empties)
text = text.apply(combine_erroneous_separations_if_page_break) 
text = text.apply(combine_erroneous_separations_if_page_break) 
text = text.apply(combine_erroneous_separations_if_page_break) 
text = text.apply(combine_erroneous_separations_if_page_break) 
text = text.apply(remove_empties)
text = text.apply(add_apostrophes)

print(min(text, key = len))


outfile = 'test3.csv'
text.to_csv(outfile)


#testing
print('After applying cleaning functions:')
test5 = text
test5 = test5.explode()
test5 = test5.loc[test5!='']
test5 = test5.reset_index(drop=True)
print(test5.head())
print(test5.shape)
print(test5.apply(lambda x : 'The recent information on inflation was seen'  in x ).sum() > 0)
#\testing

end_result['text'] = text


#Note: Do text split "by unanimous vote" here

#Breaking down document to paragraphs
Output = end_result.explode('text')
Output = Output.rename(columns={'text':'Paragraphs'})

#Defining bounds of relevant text
Output = Output.reset_index(drop=True)
Output['temp'] = Output['Paragraphs'].apply(lambda x: 'unanimous' in x or 'Unanimous' in x)
temp = Output.loc[ Output['temp'] ].groupby('ID').first()
results = Output.loc[Output['temp']].reset_index().groupby('ID').first().set_index('index')
Output['Start flag'] = False
Output['Start flag'].loc[results.index] = True

print('Unique IDs:',Output['ID'].nunique())
#TODO:
#Figure out why this is dropping 2 meetings entirely. 
#happening in start flag. 
#dropping irrelevant paragraphs
print('Shape before dropping Keep:',Output.shape)
for i in range(1, len(Output)):
    Output.loc[i,'Keep'] = Output.loc[i,'Start flag']
for i in range(1, len(Output)): 
    if Output.loc[i-1,'Keep'] == True:
        if Output.loc[i,'ID'] == Output.loc[i-1,'ID']:
            Output.loc[i,'Keep'] = Output.loc[i-1,'Keep']

Output = Output.reset_index(drop = True)
Output = Output.drop(Output[Output.Keep == False].index)

print('Shape after dropping Keep:',Output.shape)
print('Unique Keeps:',Output['Keep'].nunique())
print('Values of Keep:',Output['Keep'].unique())

print('Unique IDs:',Output['ID'].nunique())


#TODO: Add testing for this section

#dropping short paragraphs
Paragraphs_tokenized = []
Paragraphs = pd.Series(Output['Paragraphs'])
for para in Paragraphs:
    results = tokenizer.encode(para, add_special_tokens=True)
    Paragraphs_tokenized.append(results)

print('Unique IDs:',Output['ID'].nunique())


#Getting the length of the tokenized sentences
#Note: this will be overwritten later on
print('Shape before short paragraphs dropped:')
print(Output.shape)
Output['Paragraphs_tokenized'] = Paragraphs_tokenized
Output['Paragraphs_tokenized_length'] = Output['Paragraphs_tokenized'].apply(lambda x: len(x))
Output = Output.drop(Output[Output['Paragraphs_tokenized_length'] < min_length_para].index)
print('Shape after short paragraphs dropped:')
print(Output.shape)

print('Unique IDs:',Output['ID'].nunique())

#Assigns ID values to paragraphs
Output['Meta Paragraph ID'] = np.arange(len(Output))
Output['Paragraph ID'] = Output.groupby('ID').apply( lambda x: list(range(0,x.shape[0]))).explode().values

#Breaking down document to sentences
Output['Sentences'] = Output['Paragraphs'].apply(sent_tokenize)
Output = Output.explode('Sentences')


#Dropping short and long sentences.
Sentences_tokenized = []
Sentences = pd.Series(Output['Sentences'])
for sent in Sentences:
    results = tokenizer.encode(sent, add_special_tokens=True)
    Sentences_tokenized.append(results)

#Getting the length of the tokenized sentences
#Note: this will be overwritten later on
print('Shape before short/long sentences dropped:')
print(Output.shape)
Output['Sentences_tokenized'] = Sentences_tokenized
Output['Sentences_tokenized_length'] = Output['Sentences_tokenized'].apply(lambda x: len(x))
Output = Output.drop(Output[Output['Sentences_tokenized_length'] < min_length_sent].index)
Output = Output.drop(Output[Output['Sentences_tokenized_length'] > max_length_sent].index)
print('Shape after short/long sentences dropped:')
print(Output.shape)

print(Output.shape)

print('Unique IDs:',Output['ID'].nunique())

#Dropping information no longer needed for analysis
Output = Output.drop(['path','raw_text','Paragraphs_tokenized','Paragraphs_tokenized_length'],axis=1)

#Assigns ID values to sentences
Output['Meta Sentence ID'] = np.arange(len(Output))
Output['Sentence ID'] = Output.groupby('ID').apply( lambda x: list(range(0,x.shape[0]))).explode().values

print('Unique IDs:',Output['ID'].nunique())
#Testing
print('After 1 digit strings dropped:')
test7 = Output
print(test7.head())
print(test7.shape)
#\Testing
test8 = pd.Series(Output['Sentences'])
print(test8.apply(lambda x : 'Â'  in x ).sum() > 0)

outfile = 'test8.csv'
test8.to_csv(outfile)
test9 = pd.read_csv('test8.csv')
test9 = test9.dropna()
test9 = pd.Series(test9['Sentences'])
test8 = test8.reset_index(drop = True)

print(test9.apply(lambda x : 'Â'  in x ).sum() > 0)
#\Testing

print('Complete.')

outfile = 'outfile.csv'
Output.to_csv(outfile)
print('Unique IDs:',Output['ID'].nunique())

In [ ]:
print('Unique IDs:',Output['ID'].nunique())
os.chdir('/content/drive/My Drive/Thesis')
outfile = 'Output_backup.csv'
Output.to_csv(outfile)

Unique IDs: 176
